In [ ]:
# default_exp pds.ctx_index

# CTX Index

> Scraping the latest CTX index from the latest added volume.

The CTX Index is special as it's latest URL is dynamic, so it needs to be web-scraped.

In [ ]:
# hide
from nbdev.showdoc import show_doc

In [ ]:
# export
from dataclasses import dataclass
from string import Template

import pandas as pd


@dataclass
class CTXIndex:
    """Class to determine the URL for the latest cumulative index.

    This is a 2 step process, where first the MRO release page is scraped
    for the latest CTX release, and then the latest release page is scraped
    for the latest volume.
    From that latest volume the latest index URL is constructed.
    """
    volumes_url: str = "https://pds-imaging.jpl.nasa.gov/volumes/mro.html"
    release_url_template: Template = Template(
        "https://pds-imaging.jpl.nasa.gov/volumes/mro/release${release}.html"
    )
    volume_url_template: Template = Template(
        "https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_${volume}/"
    )
    scraped_tables: bool = False
    release_scraped: bool = False

    @property
    def web_tables_list(self):
        """Use the pandas scraper to read in the MRO data release table.

        The scraper returns several tables in a list and the last one
        lists all the CTX volumes.

        This could be replaced by cached properties.
        """
        if not self.scraped_tables:
            self._list_of_scraped_tables = pd.read_html(self.volumes_url)
            self.scraped_tables = True
        return self._list_of_scraped_tables

    @property
    def release_number(self):
        """Fishes out the release number.

        This is needed to construct the exact URL to the latest cumulative index file.
        """
        alist = self.web_tables_list
        return alist[-1].iloc[-1, 0].split()[-1]

    @property
    def release_url(self):
        "Constructs the release URL from the release number."
        return self.release_url_template.substitute(release=self.release_number)

    @property
    def latest_volume_url(self):
        """Scrape the Release URL for the latest volume URL in that.

        This is necessary because a release usually has more that one volume.
        """
        if not self.release_scraped:
            alist = pd.read_html(self.release_url)
            # get last row of 4th table
            row = alist[3].iloc[-1]
            number = None
            # first number that is NAN breaks the loop over last row of table
            for elem in row.values:
                try:
                    number = int(elem.split()[-1])
                except AttributeError:
                    break
            self.number = number
            self.release_scraped = True
        return self.volume_url_template.substitute(volume=self.number)

    @property
    def latest_index_label_url(self):
        "Construct the URL for the latest cumulative index."

        return self.latest_volume_url + "index/cumindex.lbl"

In [ ]:
ctx = CTXIndex()

In [ ]:
ctx.latest_index_label_url

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4044/index/cumindex.lbl'

In [ ]:
show_doc(CTXIndex.web_tables_list)

<h4 id="CTXIndex.web_tables_list" class="doc_header"><code>CTXIndex.web_tables_list</code><a href="" class="source_link" style="float:right">[source]</a></h4>

```
Use the pandas scraper to read in the MRO data release table.

The scraper returns several tables in a list and the last one
lists all the CTX volumes.

This could be replaced by cached properties.
```

In [ ]:
ctx.web_tables_list[-1].tail()

Scraping volumes page ...


,Mission to MarsContext Camera (CTX) and Mars Color Imager (MARCI) MRO LEVEL 0 EDRs V1.0,Mission to MarsContext Camera (CTX) and Mars Color Imager (MARCI) MRO LEVEL 0 EDRs V1.0.1,Mission to MarsContext Camera (CTX) and Mars Color Imager (MARCI) MRO LEVEL 0 EDRs V1.0.2
52,Release 53,NaN,CTX (33 volumes mrox_3545-3577) and MARCI (12 ...
53,Release 54,NaN,CTX (33 volumes mrox_3578-3610) and MARCI (13 ...
54,Release 55,NaN,CTX (59 volumes mrox_3611-3669) and MARCI (20 ...
55,Release 56,NaN,CTX (132 volumes mrox_3670-3801) and MARCI (33...
56,Release 57,NaN,CTX (157 volumes mrox_3802-3958) and MARCI (32...


In [ ]:
assert type(ctx.web_tables_list) == list

In [ ]:
assert len(ctx.web_tables_list) == 4

In [ ]:
show_doc(CTXIndex.release_number)

<h4 id="CTXIndex.release_number" class="doc_header"><code>CTXIndex.release_number</code><a href="" class="source_link" style="float:right">[source]</a></h4>

```
Fishes out the release number.

This is needed to construct the exact URL to the latest cumulative index file.
```

In [ ]:
assert type(ctx.release_number) == str
ctx.release_number

'57'

In [ ]:
show_doc(CTXIndex.release_url)

<h4 id="CTXIndex.release_url" class="doc_header"><code>CTXIndex.release_url</code><a href="" class="source_link" style="float:right">[source]</a></h4>

```
Constructs the release URL from the release number.
```

In [ ]:
ctx.release_url

'https://pds-imaging.jpl.nasa.gov/volumes/mro/release57.html'

In [ ]:
assert type(ctx.release_url) == str

In [ ]:
show_doc(CTXIndex.latest_volume_url)

<h4 id="CTXIndex.latest_volume_url" class="doc_header"><code>CTXIndex.latest_volume_url</code><a href="" class="source_link" style="float:right">[source]</a></h4>

```
Scrape the Release URL for the latest volume URL in that.

This is necessary because a release usually has more that one volume.
```

In [ ]:
ctx.latest_volume_url

Scraping latest release page ...


'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_3958/'

In [ ]:
assert type(ctx.latest_volume_url) == str

In [ ]:
show_doc(CTXIndex.latest_index_label_url)

<h4 id="CTXIndex.latest_index_label_url" class="doc_header"><code>CTXIndex.latest_index_label_url</code><a href="" class="source_link" style="float:right">[source]</a></h4>

```
Construct the URL for the latest cumulative index.
```

In [ ]:
ctx.latest_index_label_url

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_3958/index/cumindex.lbl'